<img src="./images/DLI_Header.png" style="width: 400px;">

# 2. HuggingFace와 PyTorch를 사용한 GPT-J의 추론


이번 실습 랩에서는 HuggingFace 라이브러리와 PyTorch를 사용하여 6B 매개 변수를 가진 큰 GPT-J 모델의 추론을 실행하는 방법에 대한 기본적인 예제를 살펴볼 것입니다. 우리는 또한 퓨샷 학습 기법을 살펴보고 GPT-J로 간단한 영어-프랑스어 번역 작업을 할 것입니다. 그 후 추론 대기 시간을 측정하겠습니다.

## 목표

이번 노트북의 목표는 다음과 같습니다.
* HuggingFace 라이브러리가 있는 하나의 GPU에서 60억 개의 매개 변수를 가진 GPT-J에 대한 간단한 추론 사례를 실행하는 방법을 이해합니다.
* 원샷 학습과 퓨샷 학습의 기본과 대규모 학습 모델의 힘을 이해합니다.
* 추론 속도를 측정해 봅니다.


**[2.1 HuggingFace에서 1 GPU로 GPU-J 6b 추론하기](#2.1)<br>**
**[2.2 퓨샷러닝](#2.1)<br>**
**[2.3 속도 측정](#2.2)<br>**

## 2.1 HuggingFace에서 1 GPU로 GPU-J 6b 추론하기

### HuggingFace에 대해서

🤗 HuggingFace Transformers 라이브러리는 텍스트, 비전 및 오디오와 같은 다양한 모달리티에서 작업을 수행하기 위해 사전 훈련된 수천 개의 모델을 제공합니다. 이 라이브러리는 HuggingFace, NVIDIA Megatron-LM 프레임워크 또는 기타 프레임워크로 사전 훈련된 대규모 GPT 모델의 추론을 지원합니다.
HF 트랜스포머를 사용하면 GPU에서 GPT-J 6b에 대한 추론을 쉽게 실행할 수 있습니다. </br>

### Eleuther.AI 와 GPT-J 6b 모델에 대해서

GPT-J 6B는 Eleuther.AI에 의해 훈련된 트랜스포머 모델입니다. Ben Wang의 Mesh Transformer JAX를 사용하는 "GPT-J"는 모델의 클래스를 의미하며, "6B"는 훈련 가능한 파라미터의 수를 나타냅니다. 모델은 모델 차원은 4096이고 피드포워드 차원은 16384인 28개의 레이어로 구성됩니다. 모델 차원은 각각 차원이 256인 16개의 헤드로 분할됩니다. RoPE(Rotary Position Embedding)는 각 헤드의 64 차원에 적용됩니다. 이 모델은 GPT-2/GPT-3와 동일한 BPE 세트를 사용하여 50257의 토큰화 어휘로 학습됩니다.

트레이닝 데이터
GPT-J 6B는 Eleuther.AI에 의해 만들어진 대규모 큐레이션 데이터 세트인 Pile에 대해 훈련되었습니다

모든 단계는 이 도커파일 https://github.com/triton-inference-server/fastertransformer_backend/blob/571a1fce438409087f5d3889237541828cc24ba5/docker/Dockerfile 로 작성된 컨테이너에서 수행됩니다.

또한 모델의 추론을 실행하기 위해 다음과 같은 파이썬 라이브러리를 설치했습니다:
- transformers==4.18.0
- huggingface_hub==0.5.1
- tokenizers==0.12.1
- SentencePiece==0.1.96
- sacrebleu==2.0.0
- jaxlib==0.3.7
- jax==0.3.7

##1 GPU로 GPT-J 6b 모델 HuggingFace 추론하기

먼저 PyTorch , Huggingface의 Transformers 와 같은 주요 라이브러리를 임포트 하겠습니다.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

사전 훈련된 GPT-J 6B 모델과 이를 위한 토크나이저를 초기화합니다. 여기 `EleutherAI`의 GPT-J 6b에 대한 전체 설명 링크가 있습니다.:  https://huggingface.co/EleutherAI/gpt-j-6B

모델이 상당히 커서 미리 다운로드하여 로컬 폴더 `weights`에 저장해 두었습니다.  모델에 대한 로컬 경로를 인자로 사용하여 `from_pretrained()`를 사용합니다. 주석처리된 라인은 HuggingFace 저장소에서 모델을 다운로드하는 방법을 보여줍니다.

<b>다음 단계에서 ` We couldn't connect to 'https://huggingface.co' to load this model` 라는 오류가 발생하거나 해당 모델/가중치를 찾을 수 없는 경우, 가중치를 백그라운드에서 다운로드 완료할 때까지 몇 분 더 기다려 주십시오. 😊</b>

In [ ]:
# model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

# We already downloaded model weights so will use local path to the weights
model = AutoModelForCausalLM.from_pretrained("./weights/gpt-j/hf")
tokenizer = AutoTokenizer.from_pretrained("./weights/gpt-j/hf")

GPU의 가용성을 확인하고 모델을 `fp16` 형식으로 GPU에 로드해 보겠습니다.  모델이 6b 가중치를 가지고 있기 때문에 `fp16` 형식을 활용해야 합니다. 필요한 GPU 메모리는 `6billion * 4bytes ~ 24GB`이지만 서버에서 사용할 수 있는 GPU는 16GB 뿐입니다. 따라서 4 바이트(fp32 형식) 대신 2 바이트(fp16 형식)를 사용하면 가중치 크기가 절반으로 줄어듭니다(`6billion * 2bytes ~ 12GB`)

.half() 함수를 사용하면 이 작업을 수행할 수 있습니다.

우리는 또한 모델을 `.eval()` 모드로 전환할 것입니다. `model.eval()` 는 훈련 및 추론(평가) 시간 동안 다르게 동작하는 모델의 일부 특정 레이어/부품들에 대한 일종의 스위치입니다. 예를 들어 드롭아웃(Dropout) 레이어, 배치놈(BatchNorm) 레이어 등이 있습니다. 우리는 모델 평가 중에 이 기능을 해제해야 하며, `.eval()` 이 자동으로 이 기능을 수행합니다. 또한 평가/검증을 위한 일반적인 방법은 `torch.no_grad()`을 `model.eval()`와 함께 사용하여 그래디언트 연산을 끄는 것입니다.

In [ ]:
assert torch.cuda.is_available()
device = torch.device("cuda:0")
model.half().to(device)
model = model.eval()

임의의 문장을 생성해 봅니다:

In [ ]:
# Generate the sentence.
with torch.no_grad():
    output = model.generate(input_ids=None, max_length=128, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

토큰에서 텍스트로 다시 디코딩하고 답을 출력할 수 있습니다:

In [ ]:
# Output the text.
for sentence in output:
    sentence = sentence.tolist()
    text = tokenizer.decode(sentence, clean_up_tokenization_spaces=True)
    print(text)

## 2.2 GPT 모델을 사용한 퓨샷 러닝 및 추론

퓨샷 러닝이란?

퓨샷 러닝은 사전 훈련된 모델이 원하는 태스크에 적응하기 위해 상대적으로 많은 양의 훈련 데이터를 필요로 하는 표준적인 파인 튜닝 기법과 달리 추론 시 몇 가지 예시와 같이 매우 적은 양의 훈련 데이터를 기계 학습 모델에 공급하여 예측을 안내하는 기법을 말합니다.

이 기술은 대부분 컴퓨터 비전에 사용되었지만, EleutherAI GPT-Neo and OpenAI GPT-3와 같은 최신 언어 모델들 중 일부와 함께 우리는 자연어 처리(NLP)에 사용할 수 있습니다.

거대 NLP 모델을 사용하여 가능하게 된 주요 기능은 퓨샷 러닝이 추론 단계에서 수행될 수 있다는 점입니다. 우리는 완성된 태스크 문장의 한 두 가지 예시와 함께 "프롬프트"를 넣어 모델에게 다음 문장에서 무엇을 기대하는지 보여줄 수 있습니다. 퓨샷 러닝은 대규모 텍스트 데이터 세트에 대한 사전 훈련 동안 광범위한 작업을 암묵적으로 수행하는 방법을 배웠기 때문에 대규모 언어 모델과 함께 사용할 수 있습니다. 이를 통해 모델은 몇 가지 예시로 관련되어 있지만 이전에 볼 수 없었던 작업을 일반화할 수 있습니다.

추론 단계에서 퓨샷 NLP의 예는 세 가지 주요 구성 요소로 구성되어 있습니다.


1. <b>태스크 설명 - Task Description</b> : 모델이 수행해야 할 작업에 대한 간단한 설명 e.g. "영어를 프랑스어로 번역해 주세요"
2. <b>예시 - Examples </b> : "sea otter => loutre de mer"와 같이 모델이 예측해야 할 것을 몇 가지 예시로 보여주기
3. <b>프롬프트 - Prompt</b> : 모델이 누락된 텍스트를 생성하여 완료해야 하는 새 예제의 시작, 예) "cheese => "

번역 작업에서 GPT-J 모델을 사용하여 퓨샷 추론을 해보겠습니다. <b>English</b>에서 <b>French</b>로 번역된 예제를 포함하는 프롬프트를 사용할 수 있으며, 문장의 마지막 부분에서는 영어 부분만 사용하고 프랑스어 부분을 비어둡니다.

<b>"English: What rooms do you have available? French:"</b>

여러 예제와 함께 이 프롬프트를 사용하여 (퓨샷), 우리는 마지막 부분에서 무엇을 볼 수 있을 것으로 예상하는지 모델에 설명하면, 모델이 마지막 문장의 프랑스어 번역으로 문장을 완성할 것을 기대할 수 있습니다.

In [ ]:
input_ids = tokenizer.encode("English: I do not speak French. French: Je ne parle pas français." \
                             "English: See you later! French: À tout à l'heure!" \
                             "English: Where is a good restaurant? French: Où est un bon restaurant?" \
                             "English: What rooms do you have available? French:", return_tensors="pt").cuda(0)

In [ ]:
# Generate translation.
output = model.generate(input_ids=input_ids, max_length=82, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

In [ ]:
sentence = output[0].tolist()
text = tokenizer.decode(sentence, clean_up_tokenization_spaces=True)
print(text)

그래서 GPT-J의 현재 대답은 다음과 같습니다: `Quel est le nombre de chambres disponibles?`

영어로는 `How many rooms are available?`라고 번역되기 때문에 가장 정확한 답은 아닙니다. 하지만 우리는 `What rooms do you have available? `라는 번역 결과를 얻기를 기대했습니다. 

이는 우리가 출력 생성 시 Greedy technique*을 사용했기 때문에 일어난 결과 입니다. 우리는 토큰별로 토큰을 생성했고, 각 생성 단계에서 토큰을 최대 확률로 가져갔습니다. 이것은 꽤 간단한 접근 방식이지만, 세대의 품질을 극대화할 수 있는 많은 다른 기술들이 존재합니다. `Beam Search`, 다양한 샘플링 기술인  `Top-K` 와 `Top-P`, 로짓(logit)의 `Temperature` 활용, 그리고 `반복 패널티(Repetition penalty)` 추가 등이 있습니다.

Greedy techique : 결과적으로 전역의 최적 솔루션으로 이어진다는 목표를  바탕으로 각 소규모 단계에서 최적의 선택을 하는 알고리즘 전략

In [ ]:
output = model.generate(input_ids=input_ids, max_length=80, num_return_sequences=1, num_beams=5, temperature=0.7, repetition_penalty=3.0, pad_token_id=tokenizer.eos_token_id)
sentence = output[0].tolist()
text = tokenizer.decode(sentence, clean_up_tokenization_spaces=True)
print(text)

더 복잡한 디코딩 방법을 사용하면 더 정확한 결과를 얻을 수 있습니다. 현재 답변인 `Quels sont vos chambres disponibles?`도 충분히 정확합니다.

디코딩 방법에 대한 자세한 내용은 아래 문서에서 확인할 수 있습니다:
[How to generate text: using different decoding methods for language generation with Transformers](https://huggingface.co/blog/how-to-generate)

## 2.3 추론 지연 시간 측정

이제 우리의 추론 파이프라인이 얼마나 빠른지 살펴보도록 하겠습니다. 우리는 PyTorch와 HuggingFace 라이브러리에서 128개의 토큰 생성 프로세스의 성능을 측정할 것입니다.

In [ ]:
# Generate the sentence.
import time

execution_time = 0
num_iterations = 10
with torch.no_grad():
    for _ in range(num_iterations):
        start = time.time()
        output = model.generate(input_ids=None, max_length=128, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id, eos_token_id=50256)
        end = time.time()
        execution_time += end - start

In [ ]:
print("Average inference time of 128 tokens is:",
     1000 * (execution_time/float(num_iterations)), "ms")

128개의 토큰이 PyTorch에서 6.3초 만에 생성될 수 있음을 알 수 있습니다. 다음 노트북으로 이동하여 최적화된 추론 파이프라인을 테스트해 보겠습니다.

<h2 style="color:green;">축하합니다!</h2>

이번 실습을 끝내느라 수고많으셨습니다! [Inference of the GPT-J 6b model with FasterTransformer](03_FTRunInferenceOfTheGPT-J.ipynb) 으로 이동하십시오.